In [1]:
import arcpy, pandas as pd, os
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist

In [11]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
css_foler = root_folder + "\\Streamflow_Sts_Drainage_Areas\GRDC_Watersheds"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"
serial_id = 'grdcno_int'

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.snapRaster = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate structure as reference for cell snap
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [12]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [13]:
# Read the Shapefile with the processed drainage areas
drain_areas = css_foler + "\\CSS-WATERSHEDS_FINAL_SELECTION.shp"

# Create a feature layer object
arcpy.MakeFeatureLayer_management(drain_areas, "drain_areas_lyr")

# Initialize an empty list to store the station IDs
sts_ids = []

# Use a SearchCursor to iterate through the rows of the feature layer
with arcpy.da.SearchCursor("drain_areas_lyr", [serial_id]) as cursor:
    for row in cursor:
        sts_ids.append(row[0])

sts_ids

[3617110,
 3617811,
 3617812,
 3617814,
 3618051,
 3618052,
 3618500,
 3618700,
 3618711,
 3618720,
 3618721,
 3618722,
 3618950,
 3618951,
 3621200,
 3623100,
 3625000,
 3626000,
 3627000,
 3628400,
 3628701,
 3628900,
 3629001,
 3630600,
 3631100,
 3632400,
 3633120,
 3633123,
 3633160,
 3633180,
 3633301,
 3634150,
 3634160,
 3634320,
 3634340,
 3634350,
 3634360,
 3634370,
 3635030,
 3635035,
 3635040,
 3635041,
 3635301,
 3635310,
 3635360,
 3635402,
 3635408,
 3635430,
 3635440,
 3635451,
 3635600,
 3635650,
 3635651,
 3637180,
 3637380,
 3637771,
 3637772,
 3637773,
 3637774,
 3637775,
 3637790,
 3637810,
 3637910,
 3638050,
 3638051,
 3638700,
 3649010,
 3649030,
 3649130,
 3649150,
 3649151,
 3649210,
 3649211,
 3649240,
 3649251,
 3649310,
 3649311,
 3649320,
 3649321,
 3649325,
 3649409,
 3649411,
 3649412,
 3649413,
 3649416,
 3649418,
 3649419,
 3649420,
 3649421,
 3649422,
 3649424,
 3649440,
 3649450,
 3649455,
 3649459,
 3649460,
 3649510,
 3649511,
 3649610,
 3649614,


In [14]:
# Create folders for other variables of tam model
raster_dir = css_foler + '\\Final_Rasters'
folders_exist([raster_dir])

In [ ]:
def get_processed_stations(raster_dir):
    """Identify station IDs that have already been processed by checking existing files"""
    processed_stations = []
    if os.path.exists(raster_dir):
        for file in os.listdir(raster_dir):
            if file.endswith("_DA.tif"):
                # Extract the station ID from the filename
                try:
                    st_id = int(file.split('_')[0])
                    processed_stations.append(st_id)
                except ValueError:
                    # Skip files that don't follow the expected naming pattern
                    continue
    return set(processed_stations)

def process_stations(sts_ids, raster_dir, serial_id):
    """Process stations with resume capability"""
    
    # Create directory if it doesn't exist
    if not os.path.exists(raster_dir):
        os.makedirs(raster_dir)
        
    # Get already processed stations
    processed = get_processed_stations(raster_dir)
    print(f"Found {len(processed)} already processed stations")
    
    # Process each station
    for st in sts_ids:
        # Skip if already processed
        if st in processed:
            print(f"\t\tSkipping Station ID: {st} (already processed)")
            continue
        
        try:
            print(f"\t\tRasterizing Drainage Area of Station ID: {st}")
            
            # Select the current station ID in the feature layer
            arcpy.SelectLayerByAttribute_management("drain_areas_lyr", "NEW_SELECTION", f"{serial_id} = {st}")

            # Create a raster from the selected feature layer
            arcpy.conversion.PolygonToRaster("drain_areas_lyr", serial_id, 
                                            raster_dir + f"\\{st}_DA.tif", 
                                            "CELL_CENTER", "NONE", env.cellSize)
            
        except Exception as e:
            print(f"Error processing station ID {st}: {str(e)}")

# Main execution
try:

    # Run the resilient processing function
    process_stations(sts_ids, raster_dir, serial_id)
    
    arcpy.CheckInExtension("spatial")
    arcpy.ClearEnvironment("workspace")
    print("\nDONE!!")
    
except Exception as e:
    # Check in extension in case of overall failure
    arcpy.CheckInExtension("spatial")
    arcpy.ClearEnvironment("workspace")
    print(f"Process interrupted: {str(e)}")
    print("You can restart the script to continue from where it left off.")

Found 507 already processed stations
		Skipping Station ID: 3617110 (already processed)
		Skipping Station ID: 3617811 (already processed)
		Skipping Station ID: 3617812 (already processed)
		Skipping Station ID: 3617814 (already processed)
		Skipping Station ID: 3618051 (already processed)
		Skipping Station ID: 3618052 (already processed)
		Skipping Station ID: 3618500 (already processed)
		Skipping Station ID: 3618700 (already processed)
		Skipping Station ID: 3618711 (already processed)
		Skipping Station ID: 3618720 (already processed)
		Skipping Station ID: 3618721 (already processed)
		Skipping Station ID: 3618722 (already processed)
		Skipping Station ID: 3618950 (already processed)
		Skipping Station ID: 3618951 (already processed)
		Skipping Station ID: 3621200 (already processed)
		Skipping Station ID: 3623100 (already processed)
		Skipping Station ID: 3625000 (already processed)
		Skipping Station ID: 3626000 (already processed)
		Skipping Station ID: 3627000 (already proce